In [11]:
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
import pickle
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from searchEngine import MySearchEngine

In [74]:
class entityDatabase:
    def __init__(self, pickle_path):
        pfile = open(pickle_path, mode='rb')
        p = pickle.load(pfile)
        #pickle_path example: "C:\\Users\\User\\Desktop\\beaver\\NumberWon\\numberwon\\entity\\test.pickle"
        self.engine = MySearchEngine(file=pickle_path,version='d')
        self.ent_dict = defaultdict(list)
        self.ent_dict = self.entize(p, self.ent_dict)
        self.ent_dict2 = defaultdict(list)
        self.ent_dict2 = self.entize2(p, self.ent_dict2)

    def get_by_id(self, id):
        return self.ent_dict[id]
    
    def entize(self, pickle, dictionary):
        #creates a dictionary where link = key, and value = list of entities
        for key, val in pickle.items():
            tokens = nltk.word_tokenize(val)
            pos = nltk.pos_tag(tokens)
            named_entities = nltk.ne_chunk(pos, binary = True)
            for i in range(0, len(named_entities)):
                ents = named_entities.pop()
                if getattr(ents, 'label', None) != None and ents.label() == "NE":
                    z = list(zip(*[ne for ne in ents]))[0]
                    z = " ".join(z)
                    dictionary[key].append(z)
        return dictionary

    def entize2(self, pickle, dictionary):
        counter = 0
        for key, val in pickle.items():
            tokens = nltk.word_tokenize(val)
            pos = nltk.pos_tag(tokens)
            named_entities = nltk.ne_chunk(pos, binary = True)
            for i in range(0, len(named_entities)):
                ents = named_entities.pop()
                if getattr(ents, 'label', None) != None and ents.label() == "NE":
                    z = list(zip(*[ne for ne in ents]))[0]
                    z = (" ".join(z), counter)
                    dictionary[key].append(z)
                counter += len(ents)
        return dictionary
          
    def searchNentity(self, qword):
        topdoc = self.engine.query(qword)[0][0]
        return self.top_entity_associated_with_item(qword,self.engine.raw_text[topdoc]) #Megan's method

    def docsearch(self, qword):
        topdoc = self.engine.query(qword)[0][0]
        raw = self.engine.raw_text[topdoc] #whole doc
        return re.match(r'(?:[^.:;]+[.:;]){1}', raw).group() #first sentence
    
    def get_title_and_first_sentence(self, qword):
        return self.engine.whats_new(qword)
    
    def top_entity_pos(self, item, most_c=10):
        #search for item.
            #for i in feed. if i == feed:
        #create a list of words that are close to word in proximity
        #score based on proximity to word.
        #documents is already a list
        word_freq = Counter()
        for i in self.ent_dict2:
            #print(self.ent_dict2[i])
            for x in self.ent_dict2[i]:
                if x[0] == item:
                    for z in self.ent_dict2[i]:
                        if x[0] != z[0]:
                            #print((abs(x[1]-z[1])))
                            word_freq[z[0]] += 1/(abs((x[1]-z[1])))
        return word_freq
        
    def top_entity_dict(self, item, most_c=10):
        #documents is already a list
        #turn each list into a counter, add all counters together. 
        mega_counter = Counter()
        for i in self.ent_dict:
            #get list of counters etc
            if item in self.ent_dict[i]:
                c = Counter(self.ent_dict[i])
                del c[item]
                mega_counter += c
        return mega_counter.most_common(most_c)

In [75]:
edatb = entityDatabase("/Users/megankaye/Desktop/BeaverWorks/Work/NumberWon/numberwon/entity/testfeed.pickle")

In [76]:
#edatb.ent_dict2

In [77]:
edatb.top_entity_pos("Katy Perry")

413
392
378
350
330
292
222
205
132
127
75
22
13
10
4
29
32
38
41
45
61
64
76
485
464
450
422
402
364
294
277
204
199
147
94
85
82
68
43
40
34
31
27
11
8
4


Counter({'American Idol': 0.0128991413761722,
         'Britain': 0.057738572574178026,
         'Calvin Harris': 0.006171910281499323,
         'FILE': 0.10730253353204174,
         'Glastonbury Festival': 0.05625,
         'Los Angeles': 0.004706192821956369,
         'MTV': 0.06057600650304224,
         'MTV Video Music Awards': 0.2576878468189444,
         'Perry': 0.03261377278184001,
         'Pop': 0.5278637770897833,
         'Pyramid Stage': 0.05925925925925926,
         'Rhythm': 0.008488157083736902,
         'Somerset': 0.05572755417956656,
         'Taylor Swift': 0.005517865219357756,
         'VMA': 0.09659364785415206,
         'Worthy Farm': 0.05664830841856806,
         'Zayn Malik': 0.0052268111035883545})

In [78]:
edatb.top_entity_dict("Katy Perry", most_c=10)

[('MTV Video Music Awards', 3),
 ('MTV', 2),
 ('Perry', 2),
 ('Pop', 2),
 ('VMA', 2),
 ('Calvin Harris', 1),
 ('Somerset', 1),
 ('Zayn Malik', 1),
 ('American Idol', 1),
 ('FILE', 1)]

In [36]:
edatb.docsearch("Trump")

'Stephen Colbert brings animated satire of Donald Trump to Showtime\n\nFILE PHOTO:'

In [37]:
edatb.engine.whats_new("Trump")

' Stephen Colbert brings animated satire of Donald Trump to Showtime\n\nFILE PHOTO: CBS television comedian Stephen Colbert rehearses a sketch on the floor of the Republican National Convention in Cleveland, Ohio, US July 17, 2016'